<a href="https://colab.research.google.com/github/fjadidi2001/Denoise/blob/main/Denoise_May18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Step 1: Set Up Environment
from google.colab import drive
drive.mount('/content/drive')

# Install dependencies
!pip install torchvision opencv-python h5py

import os
import numpy as np
import random
import h5py
import torch
import torch.nn as nn
import torch.utils.data as udata
import cv2
import glob
from torch.nn.modules.loss import _Loss
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import argparse
import time
import math
import string
from PIL import Image
import matplotlib.pyplot as plt

# Create directories
base_dir = '/content/drive/MyDrive/SSNet'
os.makedirs(base_dir, exist_ok=True)
os.makedirs(f'{base_dir}/data/VOC', exist_ok=True)
os.makedirs(f'{base_dir}/data/VOC_test', exist_ok=True)
os.makedirs(f'{base_dir}/data/Kodak24', exist_ok=True)
os.makedirs(f'{base_dir}/logos', exist_ok=True)
os.makedirs(f'{base_dir}/saved_models', exist_ok=True)
os.makedirs(f'{base_dir}/ckpts', exist_ok=True)
os.makedirs(f'{base_dir}/output', exist_ok=True)

# Step 2: Define Utility Functions (utils.py equivalent)
def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def weights_init_kaiming(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
    elif classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(mean=0, std=math.sqrt(2./9./64.)).clamp_(-0.025, 0.025)
        nn.init.constant_(m.bias.data, 0.0)

def batch_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = 0
    for i in range(Img.shape[0]):
        PSNR += peak_signal_noise_ratio(Iclean[i, :, :, :], Img[i, :, :, :], data_range=data_range)
    return PSNR / Img.shape[0]

def data_augmentation(image, mode):
    out = np.transpose(image, (1, 2, 0))
    if mode == 0:
        out = out
    elif mode == 1:
        out = np.flipud(out)
    elif mode == 2:
        out = np.rot90(out)
    elif mode == 3:
        out = np.rot90(out)
        out = np.flipud(out)
    elif mode == 4:
        out = np.rot90(out, k=2)
    elif mode == 5:
        out = np.rot90(out, k=2)
        out = np.flipud(out)
    elif mode == 6:
        out = np.rot90(out, k=3)
    elif mode == 7:
        out = np.rot90(out, k=3)
        out = np.flipud(out)
    return np.transpose(out, (2, 0, 1))

def add_text_noise(noise, occupancy=50):
    font = cv2.FONT_HERSHEY_SIMPLEX
    h, w, _ = noise.shape
    img_for_cnt = np.zeros((h, w), np.uint8)
    occupancy = np.random.uniform(0, occupancy)
    while True:
        n = random.randint(5, 10)
        random_str = ''.join([random.choice(string.ascii_letters + string.digits) for i in range(n)])
        font_scale = np.random.uniform(0.5, 1)
        thickness = random.randint(1, 3)
        (fw, fh), baseline = cv2.getTextSize(random_str, font, font_scale, thickness)
        x = random.randint(0, max(0, w - 1 - fw))
        y = random.randint(fh, h - 1 - baseline)
        color = (random.random(), random.random(), random.random())
        cv2.putText(noise, random_str, (x, y), font, font_scale, color, thickness)
        cv2.putText(img_for_cnt, random_str, (x, y), font, font_scale, 255, thickness)
        if (img_for_cnt > 0).sum() > h * w * occupancy / 100:
            break
    return noise

def add_watermark_noise(img_train, scale_lists=None, idx_lists=None, is_test=False, threshold=50):
    watermarks = []
    logo_dir = f'{base_dir}/logos'
    for ii in range(12):
        logo_path = os.path.join(logo_dir, f'{ii+1:02d}.png')
        if os.path.exists(logo_path):
            watermarks.append(Image.open(logo_path))
        else:
            print(f"Warning: Logo {logo_path} not found")
            watermarks.append(Image.new('RGBA', (100, 100)))

    img_train = img_train.numpy()
    imgn_train = img_train.copy()
    _, _, img_h, img_w = img_train.shape
    img_train = np.transpose(img_train, (0, 2, 3, 1))
    imgn_train = np.transpose(imgn_train, (0, 2, 3, 1))
    ans_scale_lists = scale_lists if scale_lists else []
    ans_idx_lists = idx_lists if idx_lists else []

    for i in range(len(img_train)):
        tmp = Image.fromarray((img_train[i] * 255).astype(np.uint8))
        img_for_cnt = Image.fromarray(np.zeros((img_h, img_w, 3), np.uint8))

        if scale_lists is None:
            scale_list = []
            idx = random.randint(0, len(watermarks)-1)
            ans_idx_lists.append(idx)
            watermark = watermarks[idx]
            w, h = watermark.size
            mark_size = np.array(watermark).size
            occupancy = threshold if is_test else np.random.uniform(0, 10)
            cnt, ratio = 0, img_w * img_h * 3 * occupancy / 100
            finish = False
            while True:
                if (ratio - cnt) < mark_size * 0.3:
                    img_train[i] = np.array(tmp).astype(np.float64) / 255.
                    break
                elif (ratio - cnt) < mark_size:
                    scale = (ratio - cnt) * 1.0 / mark_size
                    finish = True
                else:
                    scale = np.random.uniform(0.5, 1)
                scale_list.append(scale)

                water = watermark.resize((int(w * scale), int(h * scale)))
                x = random.randint(0, img_w - int(w * scale))
                y = random.randint(0, img_h - int(h * scale))
                tmp.paste(water, (x, y), water)
                img_for_cnt.paste(water, (x, y), water)
                img_cnt = np.array(img_for_cnt)
                cnt = (img_cnt > 0).sum()
                if finish:
                    img_train[i] = np.array(tmp).astype(np.float64) / 255.
                    break
            ans_scale_lists.append(scale_list)
        else:
            scale_list = scale_lists[i]
            idx = idx_lists[i]
            watermark = watermarks[idx]
            w, h = watermark.size
            for ii in range(len(scale_list)):
                scale = scale_list[ii]
                water = watermark.resize((int(w * scale), int(h * scale)))
                x = random.randint(0, img_w - int(w * scale))
                y = random.randint(0, img_h - int(h * scale))
                tmp.paste(water, (x, y), water)
            img_train[i] = np.array(tmp).astype(np.float64) / 255.

    img_train = np.transpose(img_train, (0, 3, 1, 2))
    imgn_train = np.transpose(imgn_train, (0, 3, 1, 2))
    return img_train, img_train - imgn_train, ans_scale_lists, ans_idx_lists

def add_watermark_noise_test(img, num_wm=1):
    watermarks = []
    logo_dir = f'{base_dir}/logos'
    for ii in range(12):
        logo_path = os.path.join(logo_dir, f'{ii+1:02d}.png')

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully un